# Exercise 4: Gaussian processes

## Problem 1: Moodle quiz

Answer the quiz in Moodle.

You can write below brief justifications for your answers for the yes/no questions, especially if you are unsure of your reasoning or have trouble understanding exactly what is being asked. I can then consider giving you points even if the automatic grading considered the answer wrong. You can also show the calculations you did to get the numerical answers.

In [ ]:
### Here be the justifications and calculations, if so desired


## Problem 2: GPs and marginal likelihood

You are given a data set of $x$ and $y$ pairs as defined below. Your task is to learn a good **Gaussian Process regression** model for this.

Use the RBF kernel
$$
k(x, x') = \lambda e^{-\frac{1}{2l^2} (x-x')^2}
$$
with normal likelihood
$$
p(y|f(x)) = \mathcal{N}(0, \sigma^2)
$$
where $\sigma^2$ is the noise variance.

The model has three hyperparameters, $l$, $\lambda$ and $\sigma$, and we will be inspecting their effect on the result.

Write code that:
1. Estimates the posterior distribution of the latent function $f(x)$ and evaluate it for the dense grid of test points x_test. You should do this so that it works for any choice of the hyperparameters. Note that much of this is already available in the lecture notebook.
2. Optimizes the hyperparameters by maximizing the **marginal likelihood** $p(y|x,l,\lambda,\sigma)$ of the available data. You can use gradient descent in PyTorch or some other technique (e.g. gradient descent with analytic derivatives, or even some black-box optimization routine).
3. Prints the optimal values and the resulting marginal likelihood.
4. Plots the function. Make a plot that shows (a) The mean estimate of f() for the test points (the grid over x-axis), (b) the uncertainty of f(), and (c) the uncertainty of $y$ for the test points. You should also plot the training data in the same plot so that the plots are easier to interpret.

Now use the code you wrote to study the effect of the hyperparameters, by showing the plots and marginal likelihoods for different choices of the hyperparameters. Start by showing the result with the optimal parameters and then proceed to show pairs of plots where you always make one parameter clearly smaller or larger. Label your plots clearly (with something like "Large observation noise" if using excessively large $\sigma$ etc) and **explain** how each of the parameters influences the result.


In [ ]:
# THE DATA
x = torch.tensor([ 0.0504, -1.9066, -0.5220, -1.7397,  0.5151,  1.7474,  0.2499, -1.6334,
        -1.3278, -0.6832,  0.0248, -1.6533,  1.2656, -0.4293,  1.8606, -0.6216,
         0.2714,  0.0983, -0.6295, -1.1215,  1.8418,  1.6155,  1.9598,  1.8651,
         0.3304, -1.2396,  0.6092,  1.6508,  0.3112, -0.0261])
y = torch.tensor([ 1.3866,  1.2162,  0.0608,  1.8141,  1.6938,  0.1754,  1.4721,  2.5264,
         2.4079,  0.7910,  1.1510,  1.8651,  0.9920,  0.0425,  0.4240,  0.8889,
         1.7757,  1.2457,  0.7961,  1.5359, -0.1856, -0.4273, -0.2238,  0.4841,
         1.5591,  1.8768,  1.4800,  0.4432,  1.7465,  1.7640])
N = 30

# A REGULAR GRID OVER THE X-AXIS AS TEST POINTS
x_test = torch.linspace(-2.5, 2.5, 100)


# DEFINE THE KERNEL

def covMatrix(a, b, l, lam):
    ...
    
# Compute the posterior with analytic expressions
def posterior(y_obs, k_obs, k_pred, k_pred_obs):
    ...


# Marginal likelihood
def marginal_likelihood(...):
    ...
    
# Find optimal hyperparameters
# Note that you might want to optimize over log of these if using gradient descent
l, lam, sigma = maximize_marginal_likelihood(...)


In [ ]:
# PLOTS
# Plot the latent function, its uncertainty and the uncertainty of the predictions y
# Do this for the optimal hyperparameters and also for bad configurations



## Problem 3: GPyTorch

In this exercise we practice using GPs with proper tools, rather than implementing inference from scratch.

We are given data that are pairs of univariate inputs $x$ and outputs $y$ that are constrained to be between zero and one. For instance, the input could be a dose of a drug (on some scale) and the output could be the probability that the drug has a positive effect in some population. We want to learn how $y$ depends on $x$ and realized we need a flexible model for this.

1. Start by reading the regression tutorial in https://docs.gpytorch.ai/en/latest/examples/01_Exact_GPs/Simple_GP_Regression.html. 
2. Implement the model for the data provided below (using some kernel of your choice), using **exact inference** and **normal likelihood** for the observations. Do this so that you optimize over the kernel hyperparameters and report the results with an appropriate plot.
3. Change your code to use **variational approximation** but still use the normal likelihood. Check whether you get the same (or highly similar) result as with the exact inference. The documentation has clear pointers on how to do that, for instance in https://docs.gpytorch.ai/en/latest/examples/04_Variational_and_Approximate_GPs/Non_Gaussian_Likelihoods.html. Note that you now need to set both the model parameters and the likelihood parameters as trainable, since the likelihood parameters are no longer part of the model itself.
4. Continue with the variational approximation but now change the likelihood to **Beta distribution** so that you assume $y$ to be generated from a Beta distribution with the parameters
$$
    p(y|f) = \text{Beta}(s(f)b,(1-s(f))b)
$$
where $b$ is a scale parameter, $s()$ is the sigmoid function and $f$ is a latent function following a GP prior. Note that you do not need to implement this from scratch but the library has a direct implementation for the likelihood (see https://docs.gpytorch.ai/en/latest/likelihoods.html). Compare the result of modelling the data in this way with the previous results where you assumed normal likelihood for $y$ directly. What changed?

HINTS: 
- Whenever you plot something, remember to show also the uncertainty appropriately.
- Remember to write brief written explanation for what you see. How are the results different?
- The hyperparameter optimizer may sometimes converge to a local optimum where the data is modelled purely as noise. You can try to initialize the optimization routines better to avoid this, or you can try setting priors for these parameters. See https://docs.gpytorch.ai/en/latest/examples/00_Basic_Usage/Hyperparameters.html for examples on how to do this.
- For the Beta likelihood the parameter 's' is simply one extra hyperparameter. You can either leave it at some default value or give it a some prior and let the optimizer fit it. You should not use values that are very small since the latent function would then have very little effect -- for very small $s$ we just have uniform distribution over $y$ irrespective of $f$.

In [ ]:
# OBSERVED DATA
train_x = torch.tensor([0.5126, 0.0233, 0.3695, 0.0651, 0.6288, 0.9368, 0.5625, 0.0917, 0.1680,
        0.3292, 0.5062, 0.0867, 0.8164, 0.3927, 0.9651, 0.3446, 0.5678, 0.5246,
        0.3426, 0.2196, 0.9605, 0.9039, 0.9900, 0.9663, 0.5826, 0.1901, 0.6523,
        0.9127, 0.5778, 0.4935, 0.5012, 0.6410, 0.4883, 0.9442, 0.2780, 0.2305,
        0.4002, 0.2337, 0.5585, 0.4633, 0.0507, 0.6100, 0.3050, 0.7710, 0.6987,
        0.2694, 0.3509, 0.1588, 0.9979, 0.0690, 0.7652, 0.2706, 0.2667, 0.5845,
        0.6084, 0.4865, 0.5080, 0.5669, 0.2378, 0.0224, 0.0600, 0.8233, 0.3634,
        0.9148, 0.2383, 0.9255, 0.3103, 0.7075, 0.3455, 0.6943, 0.2243, 0.9852,
        0.6694, 0.8797, 0.3302, 0.5267, 0.7142, 0.9162, 0.3036, 0.5128])
train_y = torch.tensor([0.9473, 0.9803, 0.6330, 0.9250, 0.3311, 0.9499, 0.8616, 0.9144, 0.3090,
        0.3811, 0.8733, 0.9124, 0.3459, 0.4235, 0.9813, 0.2945, 0.8940, 0.9254,
        0.3932, 0.2734, 0.9696, 0.9025, 0.9856, 0.9716, 0.8138, 0.2813, 0.2329,
        0.8965, 0.8364, 0.9311, 0.9089, 0.5626, 0.9586, 0.9882, 0.1046, 0.2527,
        0.7312, 0.2641, 0.8285, 0.8893, 0.9742, 0.4065, 0.2098, 0.1219, 0.1360,
        0.2983, 0.4822, 0.3533, 0.9649, 0.9614, 0.1990, 0.0884, 0.0490, 0.7147,
        0.6823, 0.9583, 0.8776, 0.9316, 0.0573, 0.9834, 0.9565, 0.3792, 0.6348,
        0.9224, 0.1689, 0.9594, 0.1974, 0.3019, 0.1805, 0.1727, 0.2434, 0.9776,
        0.3741, 0.8168, 0.3170, 0.9061, 0.1242, 0.8877, 0.1660, 0.9657])
N = 80

# SET OF TEST POINTS (AGAIN A GRID OVER THE INPUT AXIS)
test_x = torch.linspace(0.0, 1.0, 100)


# FOR EACH OF THE THREE MODELS YOU NEED ROUGHLY THE FOLLOWING
# 1. Specify kernel and likelihood, as well as the function that provides the marginal likelihood
# 2. Define the optimization problem: Tell what parameters are to be learned etc. Initialize with reasonable values if having trouble with convergence.
# 3. Maximize marginal likelihood
# 4. Switch to evaluation mode and make predictions for the test samples
# 5. Plot results, paying attention also to uncertainty